In [ ]:
kubectl create configmap dcobj-sample --from-file=dcobjs.json

@"
apiVersion: v1
kind: Pod
metadata:
  name: two-containers
  labels:
    environment: production
    app: nginx

# service account to read ns and pods
spec:
  serviceAccountName: cp-api-explorer

# folder served by web server
  volumes:
  - name: shared-data
    emptyDir: {}
  # sample file
  - name: dcobj-sample
    configMap:
      name: dcobj-sample

  containers:

  - name: web
    image: nginx
    volumeMounts:
    
    - name: shared-data
      mountPath: /usr/share/nginx/html

    - mountPath: /usr/share/nginx/html/sample
      name: dcobj-sample
      readOnly: true

  - name: worker
    image: ubuntu
    volumeMounts:
    - name: shared-data
      mountPath: /pod-data
    command: ["/bin/bash"]
    args:
      - "-c"
      - "date | tee /pod-data/index.html; tail -f /dev/null"
"@ | kubectl apply -f -

In [ ]:
kubectl describe pod two-containers

In [ ]:
# check if worker has timespamp for us
kubectl logs two-containers -c worker | head -2
# and web is serving it
kubectl exec two-containers -c web  -- cat /usr/share/nginx/html/index.html

In [ ]:
# expose web server on node port
kubectl expose pod two-containers --type=NodePort --port=80

In [ ]:
# access from host
minikube service list
# get url to access server
    # minikube service --url two-containers 

In [ ]:
minikube service list

In [ ]:
# clean up
kubectl delete pod two-containers
kubectl delete configmap dcobj-sample